In [ ]:
"""

機器人可以被加入群組，也可以被加入好友

我們需要依據 群組訊息 或 個人訊息 給予不同回覆

"""

In [ ]:
"""

啟用伺服器基本樣板

"""

# 引用Web Server套件
from flask import Flask, request, abort

# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)

# 載入json處理套件
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("../line_secret_key",'r',encoding="utf-8"))
server_url=secretFileContentJson.get("server_url")

# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/images" , static_folder = "../images/")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))
handler = WebhookHandler(secretFileContentJson.get("secret_key"))

# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [ ]:
"""

當群組中有人說話時，會觸發機器人

當個人單獨對機器人說話，也會觸發機器人

如果有不同業務場景，當然要分開處理囉！

"""

# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, PostbackEvent
)


@handler.add(MessageEvent, message=TextMessage)
def handle_message(event):
    print(event)
    
    if event.source.type == 'user':
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text = "說話用戶：" + event.source.user_id + 
                                    "\n這是來自個人的消息")
        )
    
    elif event.source.type == 'group':
        line_bot_api.reply_message(
            event.reply_token,
            TextSendMessage(text = "群組：" + event.source.group_id + 
                                    "\n說話用戶：" + event.source.user_id + 
                                    "\n這是來自群組的消息")
        )


In [ ]:

'''

執行此句，啟動Server，觀察後，按左上方塊，停用Server

'''

if __name__ == "__main__":
    app.run(host='0.0.0.0')